In [1]:
"""JESS"""

from pyspark.sql import SparkSession
from datetime import date
import quinn
from pyspark.sql.functions import lit


def dashes_to_underscores(s):
    return s.replace("-", "_")


#create spark session
spark = SparkSession.builder.getOrCreate()
#create df
daily_df = spark.read.csv(
    'Daily_Country_Wise_Confirmed_Cases.csv', header=True)
#perform transformations
dataset_df = daily_df.transform(
    quinn.with_columns_renamed(dashes_to_underscores))
#create pandas df and set index
pandas_df = daily_df.toPandas()
pandas_df.set_index('Country', inplace=True)
#transpose and reset index
transpose_df = pandas_df.transpose()
transpose_df.reset_index(inplace=True)
#create spark df
spark_df = spark.createDataFrame(transpose_df)
spark_df = spark_df.withColumnRenamed('index', 'Date')

# spark_df.write.csv(f"Daily_CC_{date.today()}")


22/07/08 16:01:27 WARN Utils: Your hostname, fossa-dsa-001 resolves to a loopback address: 127.0.1.1; using 192.168.0.111 instead (on interface wlp3s0)
22/07/08 16:01:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/08 16:01:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/07/08 16:01:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
spark_df.write.csv(f"Daily_CC_{date.today()}")


In [20]:
"""BRI"""


import pandas as pd
import numpy as np

# turn the spark df into pandas df
pd_pox = spark_df.toPandas()

# pd_pox = pd_pox.fillna(0)
pd.isnull(pd_pox).sum()

# drop the silly enumerated index and replace it with country names
pd_pox.set_index('Date', drop=True, inplace=True)
# this will take our row and column indices and flip 'em
# flipped_df = pd_pox.transpose()
# pd_pox.Confirmed_Cases = pd.to_numeric(pd_pox.Confirmed_Cases)
                # ('Confirmed_Cases', 'string'),
                # ('Suspected_Cases', 'string'),
                # ('Hospitalized', 'string'),
                # ('Travel_History_Yes', 'string'),
                # ('Travel_History_No', 'string'),
                # ('2022-07-01', 'string'),

# print(type(pd_pox.2022-07-01))
# )
# 
# set df data types to floats and then into ints
pd_pox = pd_pox.astype(float).astype('Int64')
# # # see if we have any null values
# pd.isnull(pd_pox).sum()
# # # replace them with 0
# flipped_df = flipped_df.fillna(0)
# pd_pox.dtypes
# pd.isnull(flipped_df).sum()
# pd_pox.columns
pd_pox.to_csv('../data/daily_cases.csv')
